In [ ]:
# OPENBIS
from pybis import Openbis
o = Openbis()
# Use this code to reconnect in case your openBIS session expires and you an error on the previous step.
sessionToken = "nqureshi-230202111003902x7B315CC0D30C1CD2D2324FDAF105573C"
o.set_token(token=sessionToken)

# INSTALL ADDITIONAL PACKAGES
!pip install --index-url https://nexus-central.leomed.ethz.ch/repository/pypi/simple mne
!pip install --index-url https://nexus-central.leomed.ethz.ch/repository/pypi/simple mne_bids
!pip install --index-url https://nexus-central.leomed.ethz.ch/repository/pypi/simple pybv==0.6.0
!pip install --index-url https://nexus-central.leomed.ethz.ch/repository/pypi/simple openpyxl
    
# INFOS
# RESTART THE SERVER IF THE FOLLOWING ERROR OCCURS
#    ValueError: events: when `type` is Stimulus, descriptions must be positve ints.
#OR
#    IndexError: list index out of range

In [ ]:
# MAIN PARAMS

raw_modes = ['BLED','LSLD'] # these recording modes will transformed into BIDS datasets
bids_path = './BIDS' # local path for temporary downloads

# METHODS
def data_slicing(save_path,photodiode_file, eeg_file, photodiode_cutoff, participant_id, project_phase, session_id, task_name):
    """ Return sliced EEG file for respective task """
    df_slice = pd.read_csv(photodiode_file)
    df_slice['f'] = df_slice['Photodiode-ch1'].astype(str).str[:2].astype(int)
    # Extracting peak value 
    peak_value = df_slice[df_slice.f >= photodiode_cutoff]   # peak value depend on visual inspection of photodiode
    peak_value.reset_index(level=0, inplace=True)
    value = peak_value.iloc[0, 0]
    initial_corrected_data = df_slice.iloc[value:,0:5]
    df_raw = pd.read_csv(eeg_file)
    result_EEG = df_raw.iloc[value:,:]
    result_EEG.to_csv(os.path.join(save_path, f"{participant_id}_{project_phase}_{session_id}_{task_name}.csv"), index = False)
    


def event_sequence(rest_time, rest_time_trial, stimulus_display, stimuli_number, event_start, datapoints_start,trial_length, sampling_frequency, data = []):
    """Return a sequence of events from a stimulation file."""
    
    # DEBUG
    #for i in range(event_start,trial_length + 1, math.floor(sampling_frequency/2)):
    #
    for i in range(event_start,trial_length + 1, math.ceil(sampling_frequency*stimulus_display)):
        data.append(i)
        
    sequence = pd.DataFrame(data)
    m = sequence.iloc[-1,:]
    #next_trial_rest = int((rest_time * sampling_frequency) + m)
    next_trial_rest = int((rest_time_trial  * sampling_frequency) + m)
    #next_trial_length = int(next_trial_rest + (stimulus_display * sampling_frequency * stimuli_number))
    next_trial_length = int(next_trial_rest + (stimulus_display * sampling_frequency * datapoints_start))
    
    return data, next_trial_rest, next_trial_length
def createBaseBidsFile(the_slice_eeg_file, layout_file, events_file, base_path = '.', unit = 'µV', sampling_frequency = 256, standard_montage = 'standard_1020', authors = ['',], funding = ''):
    """Create the base structure and files for Bids standards without additional infos."""

    df = pd.read_csv(the_slice_eeg_file)
    layout_raw = pd.read_excel(layout_file,index_col = 0)
    df_mne = df.drop(['timestamp', 'sequence', 'battery', 'flags'], axis=1)# based on bitbrain file format
    # change after Phase 1
    #df_mne["EEG"]=df_mne['EEG_ch1']
    df_mne["EEG"]=df_mne['EEG-ch1']
    #
    ch_names = []
    
    #eeg_cols = [col for col in df.columns if 'EEG' in col]
    eeg_cols = [col for col in df_mne.columns if 'EEG' in col]

    for i in range(len(eeg_cols)):
        the_name = layout_raw.index[layout_raw['Channel number'] == i+1].tolist()[0].replace('0','O')
        ch_names.append(the_name)

    data = df_mne.to_numpy().transpose()
    data = np.vstack(data/10e5) if unit == 'µV' else None
    #unit = [unit] * len(eeg_cols)
    unit = [unit] * len(eeg_cols)
    print(f"DEBUG eeg_cols: {len(eeg_cols)}")
    event = pd.read_csv(events_file)
    events = np.asarray(event)

    code = f"{events_dict['participant_id']}_{events_dict['project_phase']}_{events_dict['session_id']}_{events_dict['task_name']}"
    pybv.write_brainvision(data = data, 
                           sfreq = float(sampling_frequency), 
                           ch_names = ch_names,
                           folder_out = os.path.join(base_path,f"{code}"),
                           fname_base = code, 
                           events = events,
                           unit = unit,
                           overwrite = True)
    # change after Phase 1
    #raw = mne.io.read_raw_brainvision(os.path.join(base_path,code,f"{code}.vhdr"), preload=False)
    raw = mne.io.read_raw_brainvision(os.path.join(base_path,code,f"{code}.vhdr"), preload=False)
    raw.set_channel_types(mapping={'EOG': 'eeg'})
    raw.rename_channels(mapping={'EOG': 'EOG'})
    raw.set_channel_types(mapping={'EOG': 'eog'})
    #
    montage = mne.channels.make_standard_montage(standard_montage)
    raw = raw.copy().set_montage(montage)
    raw.set_montage(montage)
    print(raw.annotations)
    print(write_raw_bids.__doc__)
    
    bids_path = BIDSPath(subject = events_dict['participant_id'],
                         task = events_dict['task_name'],
                         root = os.path.join(base_path,code,code)
                         )
    write_raw_bids(raw, bids_path, overwrite = True)
    print(f"DEBUG path: {os.path.join(base_path,code,code)}")
    print(f"DEBUG path: {code}")
    make_dataset_description(path=os.path.join(base_path,code,code), 
                             name=code, 
                             authors = authors, 
                             funding = funding, 
                             overwrite = True)
    
    return os.path.join(base_path,f"{code}")
#
def updateBidsInfos(subject = None, task = None, suffix = None, datatype = None, root = None, extension = '.json', entries_dict = {}):
    """Add additional informations to a bids data folder."""
    
    bids_path = BIDSPath(subject = subject,
                         task = task, 
                         suffix = suffix,
                         datatype = datatype, 
                         root = root
                        )                                         
    sidecar_path = bids_path.copy().update(extension = extension)
    entries = entries_dict
    update_sidecar_json(bids_path = sidecar_path, 
                        entries = entries)
#
def updateBidsChannels(the_path, the_id, the_task, hi = 'n/a', lo = 'n/a', desc = 'EEG'):
    """Overwrite a Bids channel file with missing basic infos."""
    
    fname = os.path.join(the_path, f"sub-{the_id}", f"{desc.lower()}", f"sub-{the_id}_task-{the_task}_channels.tsv")
    df = pd.read_csv(fname, sep="\t")
    df["low_cutoff"] = lo
    df["high_cutoff"] = hi
    df["description"] = desc
    df.to_csv(fname, 
              index = False, 
              sep = "\t",
              na_rep = "n/a")
    #
def updateBidsParticipants(the_path, participant_dict):
    """Overwrite a Bids participants file with missing basic infos."""
    
    fname = os.path.join(the_path, "participants.tsv")
    df = pd.read_csv(fname, 
                     sep = "\t")
    for _k in participant_dict:
        df[_k] = participant_dict[_k]
    df.to_csv(fname, 
              index = False, 
              sep = "\t", 
              na_rep = "n/a")
    
#
def readSamplingFrequencyFromRawData(json_file, signal_name):
    """Read a Json File and return its indicated sampling frequency for a given signal name.
       Based on BitBrain file formats.
    """
    
    sampling_frequency = None
    
    with open(json_file) as jf:
        info = json.load(jf)
        for _signal in info['signals']:
            if signal_name in _signal['filename']:
                sampling_frequency = int(_signal['sampling_rate'])
    
    return sampling_frequency
#
def createEvents(save_path, data_csv_file, old_label, trial_number,rest_time_trial, sampling_frequency, trial_start, stimulus_display, datapoints_start, rest_time, stimuli_number, participant_id, project_phase, session_id, task_name):
    """Create events file for a task from a stimulation file."""
    
    df = pd.read_csv(data_csv_file)
    df.rename(columns = {f'{old_label}':'event_label'}, inplace = True)
    event_label = df['event_label']
    event_label.dropna(inplace=True)
    event_label = event_label.astype('int')
    event_start = int(sampling_frequency * trial_start)
    #trial_length = int(stimulus_display * sampling_frequency * datapoints_start)
    trial_length = int(event_start+stimulus_display * sampling_frequency * datapoints_start)
    #trial_rest = trial_length + int(rest_time * sampling_frequency)
    trial_rest = event_start
    events = event_label.to_frame()
    x = []
    
    # DEBUG
    print(f"DEBUG first event_start: {event_start}")
    rest_time = event_start
    #
    
    for _t in range(trial_number):
        x,event_start,trial_length = event_sequence(rest_time = rest_time, 
                                                    rest_time_trial=rest_time_trial,
                                                    stimulus_display = stimulus_display, 
                                                    stimuli_number = stimuli_number, 
                                                    datapoints_start=datapoints_start,
                                                    event_start = event_start, 
                                                    trial_length = trial_length, 
                                                    sampling_frequency = sampling_frequency, 
                                                    data = x)
    # DEBUG
    
    print(f"DEBUG trial rest: {trial_rest}")
    #print(f"DEBUG x len: {len(x)}")
    #print(f"DEBUG events size: {events.size}")
    #print(f"DEBUG trial_number: {trial_number}")
    #print(f"DEBUG datapoints_start: {datapoints_start}")
    #print(f"DEBUG rest_time: {rest_time}")
    #print(f"DEBUG stimulus_display: {stimulus_display}")
    #print(f"DEBUG stimuli_number: {stimuli_number}")
    print(f"DEBUG event_start: {event_start}")
    print(f"DEBUG trial_length: {trial_length}")
    #print(f"DEBUG sampling_frequency: {sampling_frequency}")
    #
    
    events['event_sequence'] = x
    events = events[['event_sequence', 'event_label']]
    events['trial'] = ""
    #events['trial'] = [i for i in range(1, trial_number + 1) for j in range(stimuli_number + 1)]
    events['trial'] = [i for i in range(1, trial_number + 1) for j in range(datapoints_start)]
    events.to_csv(os.path.join(save_path, f"{participant_id}_{project_phase}_{session_id}_{task_name}_events.csv"), index = False)
#
def createReadme(path,data_dict):
    """Create a readme file in the json format at given path and with given dict."""
    
    readme_path = os.path.join(path,"README.json")
    temp = {}
    
    with open(readme_path, "w") as outfile:
        # simplify subkeys to their attribute name
        for _k in data_dict:
            temp[_k] = {}
            for _subk in data_dict[_k]:
                the_val = data_dict[_k][_subk]
                    
                if '.' in _subk:
                    temp[_k][_subk.split('.')[1]] = the_val.split('<body><p>')[1].split('</p></body>')[0] if '<body><p>' in f"{the_val}" else the_val
                
                else:
                    #temp[_k][_subk.split[1]] = data_dict[_k][_subk]
                    temp[_k][_subk] =  the_val.split('<body><p>')[1].split('</p></body>')[0] if '<body><p>' in f"{the_val}" else the_val
           
        json.dump(temp, outfile, indent = 4)
        print(f"-- temp: {temp}")
        #json.dump(data_dict, outfile)
    
    return readme_path
#
def createBidsFiles(events_dict,the_participant):
    """Create events, raw data slicing, brainvision and bids files and return their list"""
    
    list_of_files = []
    the_participant = o.get_object(the_participant)
    
    # DEBUG
    print(f"- stim_infos: {events_dict['stim_infos']}")
    print(f"- stim.rep_time: {int(events_dict['stim_infos']['stim.rep_time'])}")
    print(f"- stim.num: {int(events_dict['stim_infos']['stim.num'])}")
    print(f"- stim.rest_time: {events_dict['stim_infos']['stim.rest_time']}")
    print(f"- stim.photodiode_cutoff: {int(events_dict['stim_infos']['stim.photodiode'])}")
    # Data slicing
    data_slicing(save_path = events_dict['save_path'], photodiode_file=events_dict['photodiode_file'], 
                 eeg_file=events_dict['raw_eeg_file'],
                 photodiode_cutoff = int(events_dict['stim_infos']['stim.photodiode']),
                 participant_id = events_dict['participant_id'], 
                 project_phase = events_dict['project_phase'], 
                 session_id = events_dict['session_id'],
                 task_name = events_dict['stim_infos']['stim.name']
                )
    the_slice_eeg_file = os.path.join(events_dict['save_path'], f"{events_dict['participant_id']}_{events_dict['project_phase']}_{events_dict['session_id']}_{ events_dict['stim_infos']['stim.name']}.csv")
    list_of_files.append(the_slice_eeg_file)
    
    # create an event file by task
    createEvents(save_path = events_dict['save_path'], 
                 data_csv_file = events_dict['stim_csv_file'],
                 
                 #old_label = events_dict['old_label'],
                 old_label = events_dict['stim_infos']['stim.label_name'],
                 
                 #trial_number = events_dict['trial_number'],
                 trial_number = int(events_dict['stim_infos']['stim.trials_number']),
                 
                 #sampling_frequency = events_dict['sampling_frequency'],
                 sampling_frequency = int(events_dict['acquisition_infos']['sensor.sampling_frequency']),
                 
                 #trial_start = events_dict['trial_start'],
                 trial_start = float(events_dict['stim_infos']['stim.trial_start']),
                 
                 #stimulus_display = events_dict['stimulus_display'],
                 stimulus_display = float(events_dict['stim_infos']['stim.display_time']) + float(events_dict['stim_infos']['stim.notdisplay_time']),
                 
                 #datapoints_start = events_dict['datapoints_start'],
                 datapoints_start = int(events_dict['stim_infos']['stim.datapoints_start']),
                 
                 #rest_time = events_dict['rest_time'],
                 rest_time = float(events_dict['stim_infos']['stim.rest_time']),
                 rest_time_trial = float(events_dict['stim_infos']['stim.rest_time']),
                 
                 #stimuli_number = events_dict['stimuli_number'],
                 stimuli_number = int(events_dict['stim_infos']['stim.rep_time']) * int(events_dict['stim_infos']['stim.num']) - 1,
                 
                 participant_id = events_dict['participant_id'], 
                 project_phase = events_dict['project_phase'], 
                 session_id = events_dict['session_id'], 
                 
                 #task_name = events_dict['task_name'],
                 task_name = events_dict['stim_infos']['stim.name']
                )
    the_events_file = os.path.join(events_dict['save_path'], f"{events_dict['participant_id']}_{events_dict['project_phase']}_{events_dict['session_id']}_{ events_dict['stim_infos']['stim.name']}_events.csv")
    list_of_files.append(the_events_file)
    # create a base bids file
    bids_root = createBaseBidsFile(#eeg_file = events_dict['raw_eeg_file'], 
                                   the_slice_eeg_file = the_slice_eeg_file,
                                   layout_file = events_dict['eeg_layout_file'],
                                   events_file = the_events_file,
                                   base_path = events_dict['save_path'], 
                                   unit = events_dict['acquisition_infos']['sensor.units'],
                                   sampling_frequency = events_dict['acquisition_infos']['sensor.sampling_frequency'], 
                                   standard_montage = events_dict['acquisition_infos']['sensor.layout'],
                                   authors = events_dict['project_infos']['proj.authors'].split(','), 
                                   funding = events_dict['project_infos']['proj.funding']
                                  )
    sub_path = f"{events_dict['participant_id']}_{events_dict['project_phase']}_{events_dict['session_id']}_{ events_dict['stim_infos']['stim.name']}"
    sub_root = os.path.join(f"{bids_root}",sub_path)
    
    # update eeg info
    updateBidsInfos(subject = events_dict['participant_id'], 
                    task = events_dict['stim_infos']['stim.name'], 
                    suffix = events_dict['acquisition_infos']['sensor.modality'].lower()[:-1], 
                    datatype = events_dict['acquisition_infos']['sensor.modality'].lower()[:-1], 
                    root = sub_root, 
                    extension = '.json', 
                    entries_dict = events_dict['acquisition_infos'],
                   )
    # update participant infos
    print(f"DEBUG the_participant.props(): {the_participant.props()}")

    temp_participant = {}
    for _k in the_participant.props():
        if 'participant.id' in _k:
            #temp_participant['participant_id'] = {the_participant.props()[_k],dict("Description":"","Levels":"")}
            #temp_participant['participant_id'] = {the_participant.props()[_k],{"Description":"","Levels":""}}
            temp_participant['participant_id'] = the_participant.props()[_k]
        elif '.' in _k:
            #temp_participant[_k.split('.')[1]] = {the_participant.props()[_k],{"Description":"","Levels":""}}
            temp_participant[_k.split('.')[1]] = the_participant.props()[_k]
            
        else:
            #temp_participant[_k] = {the_participant.props(),{"Description":"","Levels":""}}
            temp_participant[_k] = the_participant.props()[_k]
        
    
    updateBidsInfos(subject = None, 
                    task = None, 
                    suffix = 'participants', 
                    datatype = None, 
                    root = sub_root, 
                    extension = '.json', 
                    entries_dict = temp_participant, #the_participant.props(),
                   )
    #update channels infos
    updateBidsChannels(the_path = sub_root,
                       the_id = events_dict['participant_id'], 
                       the_task = events_dict['stim_infos']['stim.name'], 
                       hi = events_dict['acquisition_infos']['sensor.high_pass'], 
                       lo = events_dict['acquisition_infos']['sensor.low_pass'], 
                       desc = events_dict['acquisition_infos']['sensor.modality'].lower()[:-1]
                      )
    #update participants infos
    updateBidsParticipants(the_path = sub_root,
                           participant_dict = the_participant.props()
                          )
    #create a general description readme file
    createReadme(sub_root,{'project':events_dict['project_infos'],
                            'sensors':events_dict['acquisition_infos'],
                            'stimulation':events_dict['stim_infos']})
                               
    #zip bids folder
    #zip_path = os.path.join(sub_root,sub_path)
    zip_path = sub_root
        #zip bids folder
    #zip_path = os.path.join(sub_root,sub_path)
    zip_path = sub_root
    
    with ZipFile(f"{zip_path}.zip", mode='w') as zipf:
        len_dir_path = len(zip_path)
        for root, _, files in os.walk(zip_path):
            for file in files:
                if ".zip" not in file:
                    # rename old Readme file with Bids authors to a reference file
                    if 'README' in file and '.json' not in file:
                        old_path = os.path.join(root, file)
                        new_path = os.path.join(root,'references.txt')
                        os.rename(old_path,new_path)
                        file_path = new_path
                    elif 'references.txt' not in file:
                        file_path = os.path.join(root, file)
                    else:
                        continue
                    zipf.write(file_path, file_path[len_dir_path:])
                
    list_of_files.append(f"{zip_path}.zip")
    list_of_files.append(f"{sub_root}.eeg")
    list_of_files.append(f"{sub_root}.vhdr")
    list_of_files.append(f"{sub_root}.vmrk")
    
    return list_of_files
#
def createBidsDataset(the_bids_sample,bids_files,events_dict):
    """Create and save a new dataset from a parent sample and given files"""
    
    # create dataset for bids files
    the_dset = o.new_dataset(type = 'RAWD_BIDS', 
                             sample = the_bids_sample,
                             files = bids_files,
                             props = {f'rawd_bids.name':events_dict['task_name'],
                                      f'rawd_bids.session': events_dict['session_id'],
                                      f'rawd_bids.phase': events_dict['project_phase'],
                                      f'rawd_bids.participant': the_bids_sample.props('data_recording_mode.participant'),
                                     },
                            )
    the_dset.save()
#
def getRawFilesAndData(raw_sample,the_stim_sample,download_path):
    """Get raw and stim files from database and download them locally."""
    
    files_data = {}
    
    # get raw datasets to format
    _datasets_raw = raw_sample.get_datasets()[0] # there should be only one dataset per session
    _datasets_stim = the_stim_sample.get_datasets()[0] # idem

    print('_datasets_raw.file_list: ',_datasets_raw.file_list)
    _raw_signal = [_f for _f in _datasets_raw.file_list if '.json' in _f][0]
    _raw_eeg = [_f for _f in _datasets_raw.file_list if 'EEG.csv' in _f][0]
    _raw_eeg_layout = [_f for _f in _datasets_raw.file_list if 'Layout' in _f][0]
    _raw_photodiode = [_f for _f in _datasets_raw.file_list if 'Photodiode.csv' in _f][0]
    _raw_stim = [_f for _f in _datasets_stim.file_list if '.csv' in _f][0]
    
    # download locally the json,data file and read
    _datasets_raw.download(f"{_raw_signal}",download_path)
    _datasets_raw.download(f"{_raw_eeg}",download_path)
    _datasets_raw.download(f"{_raw_eeg_layout}",download_path)
    _datasets_raw.download(f"{_raw_photodiode}",download_path)
    _datasets_stim.download(f"{_raw_stim}",download_path)
    
    signal_info_file = f"{download_path}/{_datasets_raw.code}/original/DEFAULT/{_raw_signal.split('/')[-1]}"
    signal_info = readSamplingFrequencyFromRawData(signal_info_file, 'EEG')
    stim_file = f"{download_path}/{_datasets_stim.code}/original/DEFAULT/{_raw_stim.split('/')[-1]}"
    eeg_file = f"{download_path}/{_datasets_raw.code}/original/DEFAULT/{_raw_eeg.split('/')[-1]}"
    photodiode_file = f"{download_path}/{_datasets_raw.code}/original/DEFAULT/{_raw_photodiode.split('/')[-1]}"
    eeg_layout = f"{download_path}/{_datasets_raw.code}/original/DEFAULT/{_raw_eeg_layout.split('/')[-1]}"
    
    files_data['sampling_frequency'] = signal_info
    files_data['stim_file'] = stim_file
    files_data['eeg_file'] = eeg_file
    files_data['photodiode_file'] = photodiode_file
    files_data['eeg_layout_file'] = eeg_layout
    files_data['signals_file'] = signal_info_file
    
    return files_data
    
#
the_bids_collection = o.get_collection('/MATERIALS/NRMD/NRMD_RAWD_BIDS')
for mode in raw_modes:
    print(f"looking for raw data mode: {mode}")
    # get the main samples collection
    the_raw_collection = o.get_collection(f'/MATERIALS/NRMD/NRMD_RAWD_{mode}')
    the_raw_samples = the_raw_collection.get_samples()
    # get the recording samples
    for raw_sample in the_raw_samples:
        
        raw_parents = o.get_object(raw_sample.permId).parents
        #print(sample.code)
        sample_code = raw_sample.code.split('_')
        '''
        try:
        '''

        # check if the session is a valid one and not one of SXXX, SYYY, ...
        try:
            int(sample_code[5][1:])
        except  Exception as e:
            print(f"   - session invalid: {e}")
            continue

        # get or create the raw bids samples collection
        try:
            the_stim_sample = o.get_sample(f'/MATERIALS/NRMD/NRMD_RAWD_STIM_{sample_code[3]}_{sample_code[4]}_{sample_code[5]}')
            print(f"   - found corresponding stimulation sample: {the_stim_sample.code}")
        except  Exception as e:
            print(f"   - stimulation sample {the_stim_sample.code} not found. Skipping...")
            print(f"   - error: {e}")
            continue

        try:
            the_bids_sample = o.get_sample(f'/MATERIALS/NRMD/NRMD_RAWD_BIDS_{sample_code[3]}_{sample_code[4]}_{sample_code[5]}')
            print(f"   - getting existing data recording mode: {the_bids_sample.code}")

        except Exception as e:
            the_bids_sample = o.new_sample(type = 'DATA_RECORDING_MODE',
                                           code = f'NRMD_RAWD_BIDS_{sample_code[3]}_{sample_code[4]}_{sample_code[5]}',
                                           project = raw_sample.project,
                                           space = 'MATERIALS',
                                           collection = f'/MATERIALS/NRMD/{the_bids_collection.code}',
                                           props = {'data_recording_mode.data_recording_mode':raw_sample.props()['data_recording_mode.data_recording_mode'],
                                                    'data_recording_mode.session': raw_sample.props()['data_recording_mode.session'],
                                                    'data_recording_mode.phase': raw_sample.props()['data_recording_mode.phase'],
                                                    'data_recording_mode.participant': raw_sample.props()['data_recording_mode.participant'],
                                                   },
                                            )
            the_bids_sample.save()
            the_bids_sample.parents = raw_parents + [raw_sample,the_stim_sample]
            the_bids_sample.save()
            print(f"   - getting newly created data recording mode: {the_bids_sample.code}")


        download_path = os.path.join(bids_path,f"{sample_code[3]}_{sample_code[4]}_{sample_code[5]}")
        proj_par = [_p for _p in raw_parents if o.get_object(_p).type == 'PROJECT'][0]
        sensor_par = [_p for _p in raw_parents if o.get_object(_p).type == 'SENSOR'][0]
        stim_par = [_p for _p in raw_parents if o.get_object(_p).type == 'STIM'][0]
        participant_par = [_p for _p in raw_parents if o.get_object(_p).type == 'PARTICIPANT'][0]
        print(f"   - raw data download path: {download_path}")
        
        # check if bids dataset already exists for the task
        if len(list(the_bids_sample.get_datasets())) == 0:
            # get raw datasets to format
            try:
                infos = getRawFilesAndData(raw_sample,the_stim_sample,download_path)
            except Exception as e:
                print(f"   - raw data error: {e}")
                continue
            # create bids files by task
            events_dict = {'save_path': bids_path,
                           'stim_csv_file': infos['stim_file'],
                           'signals_file': infos['signals_file'],
                           'raw_eeg_file': infos['eeg_file'],
                           'photodiode_file':infos['photodiode_file'],
                           'eeg_layout_file': infos['eeg_layout_file'],
                           'participant_id': sample_code[3],
                           'project_phase': sample_code[4],
                           'session_id': sample_code[5],
                           'task_name': o.get_object(stim_par).props('stim.name'),
                           'project_infos': o.get_object(proj_par).props(),
                           'acquisition_infos': o.get_object(sensor_par).props(), 
                           'stim_infos': o.get_object(stim_par).props()
                          }
            print(f"   - events dict: {events_dict}")
            #if the_bids_sample.get_datasets()[0].props('rawd_bids.name')!= events_dict['task_name']:

            # all bids files to create the dataset
            bids_files = createBidsFiles(events_dict,participant_par)
            # create dataset for bids files
            createBidsDataset(the_bids_sample,bids_files,events_dict)
            # remove downloaded files
            shutil.rmtree(bids_path) #bids_path
            print(f"   - files removed from: {download_path}")
            
            #else:
            #    print(f"   - task dataset already exists. Skipping...")
        else: 
            print(f"   - task dataset already exists. Skipping...")
        '''
        except Exception as e:
            print (f"Error: {e}")
            print(f"   - session {raw_sample.code} not valid. Skipping...")
        '''

print("DONE")   